# MTH 371 Homework #2
Saeah Go \\
Due February 2, 2022 at 11:59 PM

# My tasks
Write an iterative algorithm for solving $Ax = b$ for a given sparse s.p.d matrix $A$ stored in $csr$ format. The ```function``` should take as input: \\
$\bullet$ matrix ```A```, ```b```,initial iterate ```x_0```, maximal number of iterations ```max_iter```, tolerance ```e```. \\
$\bullet$ on ouput produces: number of iterations, iter, residual norm δ = $||r||$;accuracy achieved, i.e, $\frac{||r||}{||r_0||}$, where δ = $||r_0||$ is the norm of the initial residual.

In [1]:
import numpy as np
import pandas # read the data
from scipy.linalg import lu, lu_factor, lu_solve, solve, norm
from scipy.sparse import coo_matrix, csc_matrix, random #, csr_matrix, random
from scipy.sparse.linalg import splu, spsolve, spsolve_triangular

# read the data file
def read_matrix(filepath):
    df = pandas.read_csv(filepath,
                  sep=" ",
                  header=None)
    df.columns = ["col", "row", "data"]
    mtx = coo_matrix((df["data"], (df["row"], df["col"]))).tocsr()

    return mtx

# read the data file (coordinate list)
def read_coo(filepath):
    data = pandas.read_csv(filepath,
                         sep=" ",
                         header=None)
    data.columns = ["col", "row", "data"] 
    data.col = data.col - 1 # coo matrix starts from 1, not starts from 0. so update the column indices
    data.row = data.row - 1 # similar to the column indices, update the row indices as well
    matrix = coo_matrix((data["data"], (data["col"], data["row"])))
    matrix = matrix.tocsr()

    return matrix

# LU factorization using existing package
def LU_factor(sparse_matrix):
    lu = splu(A.tocsc(),
          permc_spec="NATURAL",
          diag_pivot_thresh=0,
          options={"SymmetricMode":True})
    return lu.L , lu.U

# D(diagonal) for L1 smoother
def make_inverse_diag_mtx(A):
    n = A.shape[0]
    D = np.zeros((n,n))
    for i in range (n):
      rowsum = 0
      
      for j in range (n):
         rowsum += abs(A[i, j])
      
      D[i, i] = 1/rowsum
    return D

# forward substitution function for HW #1
def forward_subs(L, b):
    n = L.shape[0]
    y = np.zeros(n)
    for i in range(n):
      temp = b[i]
      for j in range(i+1):
        temp -= L[i,j] * y[j]
      y[i] = temp/L[i, i]
    return y

# backward substitution function for HW #2
def backward_subs(U, y):
    n = U.shape[0]
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
      temp = y[i]
      for j in range(i+1, n):
        temp -= U[i, j] * x[j]
      x[i] = temp/U[i,i]
    return x   

In [2]:
def forwardGS(A, b, x, max_iterations, tolerance):
  iter = 0
  n = A.shape[0]
  r_0 = 0 
  r = 0 
  err_rate = 1
  x_old = [0 for i in range(n)]
  x_k = x_old

  for k in range(max_iterations):
  
    for j in range(n):
      x_old  = x_k.copy() 
      x_k = forward_subs(A, b) #D + L, b)
    
    for j in range(n):
      r_0 = r_0 + np.abs(x_old[j])
      r = r + np.abs(x_k[j] - x_old[j]) 
      err_rate = abs(r/r_0) 
      if err_rate < tolerance and iter != 0: 
        return err_rate, iter
   
    z = np.linalg.inv(D) * r 

    x_k = x_k + z
   
    iter = iter + 1
  
  return err_rate, iter

In [3]:
def backwardGS(A, b, x, max_iterations, tolerance):
  iter = 0
  n = A.shape[0]
  r_0 = 0 
  r = 0 
  err_rate = 1
  x_old = [0.0 for i in range(n)]
  x_k = x_old
  
  for k in range(max_iterations):

    for j in range(n):
      x_old  = x_k.copy() 
      x_k = backward_subs(A, b) 

    for j in range(n):
      r_0 = r_0 + np.abs(x_old[j])
      r = r + np.abs(x_k[j] - x_old[j]) 
      err_rate = abs(r/r_0) 
      if err_rate < tolerance and iter != 0: 
        return err_rate, iter

    z = np.linalg.inv(D) * r 

    x_k = x_k + z

    iter = iter + 1

  return err_rate, iter

In [4]:
def symmetricGS(A, b, x, max_iterations, tolerance):

  y, iter1 = forwardGS(A, b, x, max_iterations, tolerance)
  x, iter2 = backwardGS(A, b, x, max_iterations, tolerance)

  return x, iter1 + iter2

In [5]:
# Test the solution with 25.txt
A = read_matrix('25.txt')
print(type(A))
max_iter = 100000

e = 1e-3 

x_0 = np.zeros((A.shape[0], 1)) 

b = np.ones((A.shape[0], 1))

L, U = LU_factor(A)

D = make_inverse_diag_mtx(A)
    
L_transpose = np.transpose(L)
  

residual_norm, iter = forwardGS(D + L, b, x_0, max_iter, e)
print('The accuracy achieved: ', residual_norm)
print('The number of iteration is:', iter)
residual_norm, iter = backwardGS(D + L_transpose, b, x_0, max_iter, e)
print('The accuracy achieved: ', residual_norm)
print('The number of iteration is:', iter)
residual_norm, iter = symmetricGS((D + L) @ np.linalg.inv(D) @ (D + L_transpose), b, x_0, max_iter, e)
print('The accuracy achieved: ', residual_norm)
print('The number of iteration is:', iter)


<class 'scipy.sparse.csr.csr_matrix'>
The accuracy achieved:  0.000999998397792767
The number of iteration is: 347
The accuracy achieved:  0.0009993319756690792
The number of iteration is: 346
The accuracy achieved:  0.0009995157397896365
The number of iteration is: 501


In [6]:
# Test the solution with 50.txt
A = read_matrix('50.txt') 

max_iter = 100000

e = 1e-3 

x_0 = np.zeros((A.shape[0], 1)) 

b = np.ones((A.shape[0], 1))

L, U = LU_factor(A)

D = make_inverse_diag_mtx(A)
    
L_transpose = np.transpose(L)
  

residual_norm, iter = forwardGS(D + L, b, x_0, max_iter, e)
print('The accuracy achieved: ', residual_norm)
print('The number of iteration is:', iter)
residual_norm, iter = backwardGS(D + L_transpose, b, x_0, max_iter, e)
print('The accuracy achieved: ', residual_norm)
print('The number of iteration is:', iter)
residual_norm, iter = symmetricGS((D + L) @ np.linalg.inv(D) @ (D + L_transpose), b, x_0, max_iter, e)
print('The accuracy achieved: ', residual_norm)
print('The number of iteration is:', iter)

The accuracy achieved:  0.0009990402402902134
The number of iteration is: 244
The accuracy achieved:  0.0009967487365049783
The number of iteration is: 241
The accuracy achieved:  0.0009982685016214667
The number of iteration is: 729


In [7]:
# Test the solution with 500.txt
A = read_matrix('500.txt') 

max_iter = 100000

e = 1e-3 

x_0 = np.zeros((A.shape[0], 1)) 

b = np.ones((A.shape[0], 1))

L, U = LU_factor(A)

D = make_inverse_diag_mtx(A)
    
L_transpose = np.transpose(L)
  

residual_norm, iter = forwardGS(D + L, b, x_0, max_iter, e)
print('The accuracy achieved: ', residual_norm)
print('The number of iteration is:', iter)
residual_norm, iter = backwardGS(D + L_transpose, b, x_0, max_iter, e)
print('The accuracy achieved: ', residual_norm)
print('The number of iteration is:', iter)
residual_norm, iter = symmetricGS((D + L) @ np.linalg.inv(D) @ (D + L_transpose), b, x_0, max_iter, e)
print('The accuracy achieved: ', residual_norm)
print('The number of iteration is:', iter)

KeyboardInterrupt: ignored

In [ ]:
# Test the solution with 5000.txt
A = read_matrix('5000.txt') 

max_iter = 100000

e = 1e-3 

x_0 = np.zeros((A.shape[0], 1)) 

b = np.ones((A.shape[0], 1))

L, U = LU_factor(A)

D = make_inverse_diag_mtx(A)
    
L_transpose = np.transpose(L)
  

residual_norm, iter = forwardGS(D + L, b, x_0, max_iter, e)
print('The accuracy achieved: ', residual_norm)
print('The number of iteration is:', iter)
residual_norm, iter = backwardGS(D + L_transpose, b, x_0, max_iter, e)
print('The accuracy achieved: ', residual_norm)
print('The number of iteration is:', iter)
residual_norm, iter = symmetricGS((D + L) @ np.linalg.inv(D) @ (D + L_transpose), b, x_0, max_iter, e)
print('The accuracy achieved: ', residual_norm)
print('The number of iteration is:', iter)

In [8]:
# Test the solution with 16.txt
A = read_matrix('16.txt')

max_iter = 100000

e = 1e-3 

x_0 = np.zeros((A.shape[0], 1)) 

b = np.ones((A.shape[0], 1))

L, U = LU_factor(A)

D = make_inverse_diag_mtx(A)
    
L_transpose = np.transpose(L)
  

residual_norm, iter = forwardGS(D + L, b, x_0, max_iter, e)
print('The accuracy achieved: ', residual_norm)
print('The number of iteration is:', iter)
residual_norm, iter = backwardGS(D + L_transpose, b, x_0, max_iter, e)
print('The accuracy achieved: ', residual_norm)
print('The number of iteration is:', iter)
residual_norm, iter = symmetricGS((D + L) @ np.linalg.inv(D) @ (D + L_transpose), b, x_0, max_iter, e)
print('The accuracy achieved: ', residual_norm)
print('The number of iteration is:', iter)


The accuracy achieved:  0.0009998460428669448
The number of iteration is: 622
The accuracy achieved:  0.000999794401144304
The number of iteration is: 626
The accuracy achieved:  0.000998738196106419
The number of iteration is: 1165


In [9]:
# Test the solution with 64.txt
A = read_matrix('64.txt')

max_iter = 100000

e = 1e-3 

x_0 = np.zeros((A.shape[0], 1)) 

b = np.ones((A.shape[0], 1))

L, U = LU_factor(A)

D = make_inverse_diag_mtx(A)
    
L_transpose = np.transpose(L)
  

residual_norm, iter = forwardGS(D + L, b, x_0, max_iter, e)
print('The accuracy achieved: ', residual_norm)
print('The number of iteration is:', iter)
residual_norm, iter = backwardGS(D + L_transpose, b, x_0, max_iter, e)
print('The accuracy achieved: ', residual_norm)
print('The number of iteration is:', iter)
residual_norm, iter = symmetricGS((D + L) @ np.linalg.inv(D) @ (D + L_transpose), b, x_0, max_iter, e)
print('The accuracy achieved: ', residual_norm)
print('The number of iteration is:', iter)

The accuracy achieved:  0.000998490956578548
The number of iteration is: 462
The accuracy achieved:  0.0009991269400999759
The number of iteration is: 462
The accuracy achieved:  0.0009967998164359237
The number of iteration is: 591


In [ ]:
# Test the solution with 1024.txt
A = read_matrix('1024.txt')

max_iter = 100000

e = 1e-3 

x_0 = np.zeros((A.shape[0], 1)) 

b = np.ones((A.shape[0], 1))

L, U = LU_factor(A)

D = make_inverse_diag_mtx(A)
    
L_transpose = np.transpose(L)
  

residual_norm, iter = forwardGS(D + L, b, x_0, max_iter, e)
print('The accuracy achieved: ', residual_norm)
print('The number of iteration is:', iter)
residual_norm, iter = backwardGS(D + L_transpose, b, x_0, max_iter, e)
print('The accuracy achieved: ', residual_norm)
print('The number of iteration is:', iter)
residual_norm, iter = symmetricGS((D + L) @ np.linalg.inv(D) @ (D + L_transpose), b, x_0, max_iter, e)
print('The accuracy achieved: ', residual_norm)
print('The number of iteration is:', iter)